In [2]:
from pydantic import BaseModel
from ollama import generate

## Tools

In [ ]:
def detect_objects(image: str, prompt: str) -> dict:
    pass



In [25]:
SYSTEM_PROMPT = """
You are a helpful assistant tasked with answering my queries with respect to geospatial data.
You are connecting to a GroundingDINO+SAM based tool that can help you with this task.

Your capabilities include:
1. object detection (for trees)
2. image segmentation (for trees)
3. image classification (for trees)
4. image density map generation (for trees)

the model is only trained with "coconut" and "mango" classes.

when you want to generate a certain response, call the tool with the relevant prompt.

for example,
1.
user: identify the trees in the image
assistant: find_objects("coconut . mango .")

2.
user: identify coconut trees in the image
assistant: find_objects("coconut .")

3.
user: find number of coconut trees in the image
assistant: find_objects("coconut .")

All tools will return answers in the following format:
{
    "response_image": "<file urlpath>",
    "total_count": <int>,
    "label_counts": {
        "coconut tree": <int>,
        "banyan tree": <int>
    },
    "mean_confidence": <float>
}

You have in your possession a few tools that can help you with this task.


The tools are:

1. `find_objects(image_path, prompts)`:
    Object detection tool that can identify boxes around objects in an image.
    It takes in two parameters:
    - image_path: The path to the image file.
    - prompts: The classes of objects you want to detect in the image

2. `segment_objects(image_path, prompts)`:
    Image segmentation tool that can identify pixels of objects in an image.
    It takes in two parameters:
    - image_path: The path to the image file.
    - prompts: The classes of objects you want to segment in the image

3. `density_map(image_path, prompts)`:
    Image density map generation tool that can identify the density of objects in an image.
    It takes in two parameters:
    - image_path: The path to the image file.
    - prompts: The classes of objects you want to generate a density map for

"""

SYSTEM_PROMPT = """You are a vision-language agent that helps users analyze images using a detection tool.

the model is ONLY trained with "coconut" and "mango" classes.

You have access to the following function:
---
detect_objects(image_path: string, object_class: string) → Detects all objects matching the given class in the image and returns metadata (bounding boxes, counts, confidence scores).
---

Respond according to the following rules:

1. If the user is asking you to **detect** or **find** objects in an image, return only a JSON object in this format:
{
  "tool_call": "detect_objects",
  "args": {
    "image_path": "string",          // full path to the image
    "object_class": "string"         // e.g., "coconut tree", "car", etc.
  }
}

Examples:

1. identify coconut trees in the image
{
  "tool_call": "detect_objects",
  "args": {
    "image_path": "/path/to/image.jpg",
    "object_class": "coconut ."
  }
}

2. find all trees in the image
{
  "tool_call": "detect_objects",
  "args": {
    "image_path": "/path/to/image.jpg",
    "object_class": "coconut . mango ."
  }
}

3. find number of coconut trees in the image
{
  "tool_call": "detect_objects",
  "args": {
    "image_path": "/path/to/image.jpg",
    "object_class": "coconut ."
  }
}

Always follow JSON formatting precisely when invoking a tool. Do not include any explanation or additional commentary unless the user is asking a follow-up question.
"""

class ResponseModel(BaseModel):
    tool_name: str
    tool_kwargs: dict[str, str]

rsp = generate(
    model="llama3.2:latest",
    prompt="Can you identify all objects in this image? /Users/shreyas/Downloads/groundingdino/test_images/image_0001.jpg",
    system=SYSTEM_PROMPT,
    format=ResponseModel.model_json_schema(),)

rsp

GenerateResponse(model='llama3.2:latest', created_at='2025-05-12T09:05:40.385159Z', done=True, done_reason='stop', total_duration=736675709, load_duration=35330417, prompt_eval_count=404, prompt_eval_duration=153398333, eval_count=27, eval_duration=547391833, response='{\n  "tool_name": "detect_objects",\n  "tool_kwargs": {\n    "object_class": "."\n  }\n}', context=[128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 271, 2675, 527, 264, 11376, 44658, 8479, 430, 8779, 3932, 24564, 5448, 1701, 264, 18468, 5507, 382, 1820, 1646, 374, 27785, 16572, 449, 330, 1030, 27357, 1, 323, 330, 76, 5281, 1, 6989, 382, 2675, 617, 2680, 311, 279, 2768, 734, 512, 11192, 62343, 25915, 10281, 2703, 25, 925, 11, 1665, 4895, 25, 925, 8, 11651, 34387, 82, 682, 6302, 12864, 279, 2728, 538, 304, 279, 2217, 323, 4780, 11408, 320, 62679, 15039, 11, 14921, 11, 12410, 12483, 4390, 45464, 66454, 4184, 311, 279, 2768, 5718, 1473, 16, 13, 1442, 279, 1217, 374, 10371, 499, 311, 3146, 62343,

In [26]:
print(rsp.response)

{
  "tool_name": "detect_objects",
  "tool_kwargs": {
    "object_class": "."
  }
}
